In [1]:
import torch
import numpy as np
import torch.nn as nn
import os
import sys

In [2]:
notebook_path = os.getcwd()
parent_dir = os.path.dirname(notebook_path)
sys.path.append(parent_dir)

In [3]:
import importnb
with __import__('importnb').Notebook(): 
    from tools import ScaledDotProductAttention
    from tools import MultiHeadAttention
    from tools import AddPositionalEncoding
    from tools import TransformerFFN
    from notebooks.model import TransformerEncoderLayer,TransformerEncoder
    from notebooks.model import TransformerDecoderLayer
    from notebooks.model import Transformer

## SelfAttentionの実行

### 適当なデータの作成
---
新しくデータkを作る。kは**(B,T,d_model)**のshapeを持つテンソルである。


### SelfAttentionにする
---
q,k,vを同じテンソルにすることでSelfAttentionにする。


### forwardで計算を行う。
---
呼び出したspa.forward()によって計算を行う。
この時spaの初期化で与えるd_kには本来d_modelをhead数で割った値が入る(デフォルトだと512/8で64)

In [4]:
###SelfAttentionの実行
import json
with open('../data/config.json', 'r') as file:
    config = json.load(file)
    
max_len = config["max_len"]
src_vocab_size = config["src_vocab_size"]
tgt_vocab_size = config["tgt_vocab_size"]
batch_size = 16
num_head = 8
d_model = 512
d_ff = 2048
N = 6
pad_idx = 0
dropout_rate=0.1
layer_norm_eps = 1e-5
src = torch.randint(0,2000, (batch_size,max_len)).to(torch.int64)
tgt = torch.randint(0,2000, (batch_size,max_len)).to(torch.int64)

In [13]:
q.shape

torch.Size([16, 159])

## MultiHeadAttentionの実行
ランダムなテンソル、スタンダードなマスクを使う。

In [8]:
def create_incremental_mask(seq_len):
    """
    seq_len x seq_len のサイズのマスクを生成する。
    0列目は全てFalse、以降の列では上から順にTrueの数を増やしていく。
    """
    # seq_len x seq_len の行列を生成し、初期値は全てFalseに設定
    mask = torch.full((seq_len, seq_len), False)

    # 各列に対して、上から順にTrueをセットする
    for i in range(seq_len):
        mask[:i, i] = True

    return mask
self_mask = create_incremental_mask(seq_len).repeat(batch_size,1,1)
tgt_mask = create_incremental_mask(seq_len).repeat(batch_size,1,1)
print(self_mask.shape)
print(tgt_mask.shape)

torch.Size([2, 8, 8])
torch.Size([2, 8, 8])


In [9]:
encoder = TransformerEncoder(
    d_model = d_model,
    d_ff = d_ff,
    num_head = num_head,
    N=N,
    max_len=max_len,
    pad_idx=pad_idx,
    vocab_size=vocab_size,
    dropout_rate=dropout_rate,
    layer_norm_eps=layer_norm_eps
)

In [11]:
output = encoder(
    k.to(torch.int32),
    mask=self_mask)

torch.Size([2, 8, 64])


In [16]:
decoderlayer = TransformerDecoderLayer(
    d_model=d_model,
    d_ff=d_ff,
    num_head=num_head,
    dropout_rate=dropout_rate,
    layer_norm_eps=layer_norm_eps
)

In [5]:
transformer = Transformer(
    src_vocab_size=src_vocab_size,
    tgt_vocab_size=tgt_vocab_size,
    max_len=max_len
)

In [6]:
output = transformer(
    src=src,
    tgt=tgt
)

output.shape

## LSHアルゴリズムの実装

In [16]:
num_of_hash = 4
d_model = 3

### あらかじめx_1,2,3の類似度をそれぞれ測っておく

In [78]:
from sklearn.metrics.pairwise import cosine_similarity as similarity
def generate_sample_data(d_model,num_of_hashes):
    ### 適当なベクトルを三つ生成
    x_1 = torch.randn(d_model)
    x_2 = torch.randn(d_model)
    x_3 = torch.randn(d_model)
    pairs = [(x_1,x_2,"1-2"),(x_1,x_3,"1-3"),(x_2,x_3,"2-3")]
    similarities = similarities = [(pair[2], similarity(pair[0].reshape(1,-1), pair[1].reshape(1,-1))) for pair in pairs]
    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
    print(similarities)
    ### 標準正規分布に従う行列を作成
    R = np.random.randn(d_model,int(num_of_hash/2))
    return ((x_1,x_2,x_3),R)

In [116]:
vectors,R = generate_sample_data(d_model,num_of_hash)

[('1-2', array([[-0.2635013]], dtype=float32)), ('1-3', array([[-0.41672304]], dtype=float32)), ('2-3', array([[-0.7318527]], dtype=float32))]


In [117]:
def sample_lsh(input,R):
    x_R = input@R
    x_R = torch.cat([x_R, -x_R], dim=1)
    ## argmaxに書き換えてもいいけどmax使った方がhashの値まで見えていいかなと思ってこっちにしてる
    hash = torch.max(x_R,dim=1).indices
    return hash

In [176]:
a = torch.vstack(
    [
        vectors[0],
        vectors[1],
        vectors[2]
    ])

TypeError: vstack() got an unexpected keyword argument 'requires_grad'

In [119]:
a

tensor([[ 2.3341, -1.9627,  0.5752],
        [ 0.3347,  0.6775, -1.7282],
        [-0.8408,  0.4555,  1.3133]])

In [120]:
labels = sample_lsh(a,R)

In [121]:
labels

tensor([2, 1, 0])

In [122]:
a

tensor([[ 2.3341, -1.9627,  0.5752],
        [ 0.3347,  0.6775, -1.7282],
        [-0.8408,  0.4555,  1.3133]])

In [123]:
sorted_indices = labels.argsort()
sorted_data = a[sorted_indices]

In [124]:
sorted_data

tensor([[-0.8408,  0.4555,  1.3133],
        [ 0.3347,  0.6775, -1.7282],
        [ 2.3341, -1.9627,  0.5752]])

In [165]:
from torch import nn
class Calculate_LSH(nn.Module):
    def __init__(
        self,
        d_model:int,
        num_length:int,
        num_buckets:int,
    )->None:
        super().__init__()
        ## define random matrix for rotation
        self.R = torch.randn(size = (d_model,int(num_buckets/2)))
        ## define vectors to store labels
        ## ここは別にハッシュ計算部分で個別に定義してもいいけど変換を可逆にしたい時に便利かなって思ったのでこうしてます
        self.labels = torch.randint(high=num_buckets,size=(num_length,))
    def forward(
        self,
        input:torch.tensor,
    )-> torch.tensor:
        ##requires->行列
        ##effects->ハッシュを元に各ベクトルのラベルを求め、ラベルごとに行列をソート
        self.labels = self.__cal_hash(input)
        sorted_indices = self.labels.argsort()
        input = torch.index_select(input, 0 , sorted_indices)
        return input
    def __cal_hash(
        self,
        input:torch.tensor,
    ):
        ## requires->行列
        ## effects->ハッシュを計算、単語ごとにハッシュを割り当ててラベルベクトルとして返す。
        x_R = input@self.R
        x_R = torch.cat([x_R, -x_R], dim=1)
        ## argmaxに書き換えてもいいけどmax使った方がhashの値まで見えていいかなと思ってこっちにしてる
        hash = torch.max(x_R,dim=1).indices
        return hash

In [166]:
model = Calculate_LSH(
    d_model = 3,
    num_length = 3,
    num_buckets = 4,
)

In [183]:
a = torch.rand(size=(3,3),dtype=torch.float32,requires_grad=True)

In [184]:
a

tensor([[0.4981, 0.3462, 0.0865],
        [0.9151, 0.9980, 0.9033],
        [0.3711, 0.4656, 0.9315]], requires_grad=True)

In [185]:
print(model(a))

tensor([[0.4981, 0.3462, 0.0865],
        [0.9151, 0.9980, 0.9033],
        [0.3711, 0.4656, 0.9315]], grad_fn=<IndexSelectBackward0>)


In [186]:
model.labels

tensor([2, 3, 3])

In [188]:
model(a).requires_grad

True